<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_03_2_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 3: Large Language Models**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Foundation Models [[Video]]() [[Notebook]](t81_559_class_03_1_llm.ipynb)
* **Part 3.2: Text Generation** [[Video]]() [[Notebook]](t81_559_class_03_2_text_gen.ipynb)
* Part 3.3: Text Summarization [[Video]]() [[Notebook]](t81_559_class_03_3_text_summary.ipynb)
* Part 3.4: LLM Writes a Book [[Video]]() [[Notebook]](t81_559_class_03_4_book.ipynb)
* Part 3.5: Small Large Language Models [[Video]]() [[Notebook]](t81_559_class_03_5_small_llm.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
  Att

# 3.2: Text Generation

Text generation is one of the most common tasks for LLMs. We've already seen how to use the LLM to generate code; generating regular text for human consumption is similar. To generate text, we will not use a conversational chat style; instead, we will send prompts to LangChain and receive the generated text.

We use the following code to query the LLM for text generation.




In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown

MODEL = 'gpt-3.5-turbo'
TEMPERATURE = 0.2

def get_response(llm, prompt):
  messages = [
      SystemMessage(
          content="You are a helpful assistant that answers questions accurately."
      ),
      HumanMessage(content=prompt),
  ]

  print("Model response:")
  output = llm.invoke(messages)
  display_markdown(output.content, raw=True)

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature=TEMPERATURE,
  n= 1,
  max_tokens= 256)

## Text Generation Patterns

For simple text generation, you will see several different prompting patterns. These patterns vary depending on the amount of information you provide the LLM. The patterns we will examine in this module are listed here.

* Zero-Shot
* One-Shot
* Few-Shot


## Zero-Shot Text Generation

A zero-shot prompt for text generation is a method where you provide a language model with a single prompt to generate text, without any prior fine-tuning or specific training on related tasks. To use this approach effectively, you should craft a detailed and clear prompt that communicates exactly what you want the model to generate. Include the type of content, style, and any specific information or constraints that are important to the task. For instance, if you're asking for a business email, you might specify the tone (formal or informal), the main points to cover (meeting time, purpose, attendees), and any call to action. The key is to be explicit about the desired output to guide the model's response accurately, as it relies solely on the information provided in the prompt to produce relevant and coherent text. This method is highly versatile and can be applied across various text generation tasks without the need for customized training.

The following text is an example of a zero-shot prompt. I make many requests and provide information about the student, but I do not give the LLM a sample to work from.

In [3]:
print(get_response(llm, """
Generate a positive letter of reccomendation for John Smith, a student of mine
for INFO 558 at Washington University, my name is Jeff Heaton. He is applying
for a Master of Science in Computer Science. Just give me the
body text of the letter, no header or footer. Format in markdown.
Below is his request.

I hope this message finds you well and that you are enjoying the holiday season!
I am John Smith (ID: 1234), a proud alumnus of WashU, having graduated in
January 2021 with a Master’s degree in Quantitative Finance.

During the spring semester of 2020, I had the pleasure of attending your course,
INFO 558: Applications of Deep Neural Networks, which was an elective for my
master's program. I thoroughly enjoyed the content and was deeply engaged
throughout, culminating in an A+ grade.

Since graduating with a 3.99 GPA—top of my major—I have been working as a Senior
Financial Risk Analyst at RGA. My role primarily involves developing automation
tools and programming for strategic analysis and other analytical tasks. To
further enhance my programming skills and knowledge, I am planning to pursue a
part-time Master's in Computer Science while continuing to work at RGA.

I am a great admirer of your work (I’m a regular viewer of your YouTube channel
and have recommended it to my colleagues), and your insights would be invaluable
in my application. I am applying to the following programs:

Georgia Tech, Master of Science in Computer Science
University of Pennsylvania, Master of Computer & Information Technology
Could I possibly ask for your support with a recommendation letter for these
applications? I have attached my resume for your reference and am happy to
provide any additional information you might need.

Thank you very much for considering my request. I look forward to your
positive response.

Warm regards,

John
"""))

Model response:


```markdown
Dear Admissions Committee,

It is with great pleasure that I write this letter of recommendation for John Smith, a former student of mine at Washington University in St. Louis. I had the privilege of teaching John in INFO 558: Applications of Deep Neural Networks during the spring semester of 2020, where he demonstrated exceptional dedication, intelligence, and a strong work ethic.

John's academic performance in my course was outstanding, earning him an A+ grade. His enthusiasm for the subject matter was evident in his active participation in class discussions and his ability to grasp complex concepts quickly. John consistently displayed a high level of analytical thinking and problem-solving skills, which are essential qualities for success in the field of Computer Science.

Since graduating with a Master's degree in Quantitative Finance from WashU, John has excelled in his role as a Senior Financial Risk Analyst at RGA. His responsibilities involve developing automation tools and programming for strategic analysis, showcasing his proficiency in coding and data analysis. John's decision to pursue a part-time Master's in Computer Science while working full-time is a testament to his commitment to continuous learning and professional growth.

I have no doubt that John's passion for technology, coupled with his strong academic background and practical experience, make him an ideal candidate for the

None


## One-Shot Text Generation

A one-shot prompt for text generation is a technique where you provide a single, detailed input to a language model to generate text based on that prompt. To use this effectively, start by crafting a clear and concise prompt that includes all necessary details and context needed for the output you desire. Specify the style, tone, and specific elements you want to include. For example, if you want a descriptive paragraph about a seaside town, mention key details like the time of day, the atmosphere, and any particular imagery or emotions you want to evoke. This precision helps the model understand your expectations and produce more relevant and focused content. Once you've prepared your prompt, simply input it into the text generation tool and evaluate the generated text, tweaking your prompt as needed to refine the results.

In [4]:
print(get_response(llm, """
Generate a positive letter of reccomendation for John Smith, a student of mine
for INFO 558 at Washington University, my name is Jeff Heaton. He is applying
for a Master of Science in Computer Science. Just give me the
body text of the letter, no header or footer. Format in markdown.

-----------------
This is an example letter of reccomendation, written by me.

To Whom It May Concern:
John earned an A+ in my course Applications of Deep Neural Networks for the
Fall 2019 semester at Washington University in St. Louis. During the semester
I got a chance to know John through several discussions, both about my course
and his research interests. While John did not come from a computer science
background he has demonstrated himself as a capable Python programmer and was
able to express his ideas in code.  My primary career is as a VP of data science
at RGA, a Fortune 500 insurance company.  In this role I know the value of
individuals, such as John, who have a background in finance, understand
advanced machine learning topics, and can code sufficiently well to function
as a data scientist.

-----------
The details of this student's request follows.

I hope this message finds you well and that you are enjoying the holiday season!
I am John Smith (ID: 1234), a proud alumnus of WashU, having graduated in
January 2021 with a Master’s degree in Quantitative Finance.

During the spring semester of 2020, I had the pleasure of attending your course,
INFO 558: Applications of Deep Neural Networks, which was an elective for my
master's program. I thoroughly enjoyed the content and was deeply engaged
throughout, culminating in an A+ grade.

Since graduating with a 3.99 GPA—top of my major—I have been working as a Senior
Financial Risk Analyst at RGA. My role primarily involves developing automation
tools and programming for strategic analysis and other analytical tasks. To
further enhance my programming skills and knowledge, I am planning to pursue a
part-time Master's in Computer Science while continuing to work at RGA.

I am a great admirer of your work (I’m a regular viewer of your YouTube channel
and have recommended it to my colleagues), and your insights would be invaluable
in my application. I am applying to the following programs:

Georgia Tech, Master of Science in Computer Science
University of Pennsylvania, Master of Computer & Information Technology
Could I possibly ask for your support with a recommendation letter for these
applications? I have attached my resume for your reference and am happy to
provide any additional information you might need.

Thank you very much for considering my request. I look forward to your
positive response.

Warm regards,

John
"""))

Model response:


```markdown
To Whom It May Concern:

I am pleased to write this letter of recommendation for John Smith, a former student of mine in INFO 558: Applications of Deep Neural Networks at Washington University in St. Louis. John excelled in my course, earning an A+ grade during the Fall 2019 semester.

Throughout the semester, I had the opportunity to engage with John in discussions related to the course material and his research interests. Despite not having a background in computer science, John demonstrated strong Python programming skills and the ability to effectively translate his ideas into code. As a VP of data science at RGA, a Fortune 500 insurance company, I recognize the value of individuals like John who possess a blend of finance knowledge, expertise in advanced machine learning concepts, and proficient coding abilities essential for roles in data science.

John's dedication to learning and his impressive academic performance make him a standout candidate for the Master of Science in Computer Science program. I have no doubt that he will continue to excel in his academic pursuits and contribute positively to any program he chooses to undertake.

I highly recommend John for admission to the Master of Science in Computer Science program at Georgia Tech and the Master of Computer & Information Technology program at the University of Pennsylvania. His passion for learning, strong

None


## Few-Shot Text Generation

A few-shot prompt involves presenting a model with a small set of examples to guide its behavior in generating responses or predictions. This technique is particularly useful in machine learning models like language or image generation systems, where the prompt acts as a mini-training session, enabling the model to understand and replicate a desired pattern or style with limited input. For instance, in a text generation model, a few-shot prompt might include a handful of sentences along with the desired outputs, setting the stage for the model to continue producing similar results. This approach helps in refining the model's outputs without the need for extensive training data, making it adaptable and efficient for specific tasks or creative nuances.

In [5]:
print(get_response(llm, """
Generate a positive letter of reccomendation for John Smith, a student of mine
for INFO 558 at Washington University, my name is Jeff Heaton. He is applying
for a Master of Science in Computer Science. Just give me the
body text of the letter, no header or footer. Format in markdown.

-----------------
Examples of letters of reccomendation, written by me.

To Whom It May Concern:
John earned an A+ in my course Applications of Deep Neural Networks for the
Fall 2019 semester at Washington University in St. Louis. During the semester
I got a chance to know John through several discussions, both about my course
and his research interests. While John did not come from a computer science
background he has demonstrated himself as a capable Python programmer and was
able to express his ideas in code.  My primary career is as a VP of data science
at RGA, a Fortune 500 insurance company.  In this role I know the value of
individuals, such as John, who have a background in finance, understand
advanced machine learning topics, and can code sufficiently well to function
as a data scientist.

John was a student that in my class, T81-558: Application of Deep Neural Networks,
for the Spring 2017 semester. This is a technical graduate class which includes
students from the Masters of Science lnformation Systems, Management,
computer science, and other disciplines. The course teaches students to
implement deep neural networks using Google TensorFlow and Keras in the Python
programming language. Students are expected to complete four computer programs
and complete a final project. John did well in my course and earned an A+ (4.0).

-----------
The details of this student's request follows.

I hope this message finds you well and that you are enjoying the holiday season!
I am John Smith (ID: 1234), a proud alumnus of WashU, having graduated in
January 2021 with a Master’s degree in Quantitative Finance.

During the spring semester of 2020, I had the pleasure of attending your course,
INFO 558: Applications of Deep Neural Networks, which was an elective for my
master's program. I thoroughly enjoyed the content and was deeply engaged
throughout, culminating in an A+ grade.

Since graduating with a 3.99 GPA—top of my major—I have been working as a Senior
Financial Risk Analyst at RGA. My role primarily involves developing automation
tools and programming for strategic analysis and other analytical tasks. To
further enhance my programming skills and knowledge, I am planning to pursue a
part-time Master's in Computer Science while continuing to work at RGA.

I am a great admirer of your work (I’m a regular viewer of your YouTube channel
and have recommended it to my colleagues), and your insights would be invaluable
in my application. I am applying to the following programs:

Georgia Tech, Master of Science in Computer Science
University of Pennsylvania, Master of Computer & Information Technology
Could I possibly ask for your support with a recommendation letter for these
applications? I have attached my resume for your reference and am happy to
provide any additional information you might need.

Thank you very much for considering my request. I look forward to your
positive response.

Warm regards,

John
"""))

Model response:


Dear Admissions Committee,

I am writing to highly recommend John Smith for admission to the Master of Science in Computer Science program at your esteemed institution. I had the pleasure of teaching John in my course, INFO 558: Applications of Deep Neural Networks, at Washington University in St. Louis. Throughout the semester, John consistently demonstrated exceptional dedication, intelligence, and a strong work ethic.

Despite not having a background in computer science, John quickly adapted and excelled in the course, earning an outstanding A+ grade. His ability to grasp complex concepts, coupled with his proficiency in Python programming, was truly impressive. John's passion for learning and his commitment to excellence were evident in his performance and interactions within the class.

Having graduated with a Master's degree in Quantitative Finance and excelling in his professional role as a Senior Financial Risk Analyst at RGA, John has proven himself to be a highly motivated and capable individual. His decision to pursue further education in Computer Science speaks volumes about his ambition and determination to broaden his skill set.

I have no doubt that John's analytical skills, programming expertise, and strong academic background make him an ideal candidate for your program. His enthusiasm for learning, coupled with his ability to apply theoretical knowledge to practical scenarios, sets him apart as a promising candidate who

None
